<h1>Example Langchain with MSSQL Server</h1>

<h4>Initialize the DB</h4>

In [ ]:
!pip install pyodbc

In [1]:
# from langchain.sql_database import SQLDatabase
from Custom_SqlDatabase import SQLDatabase

# Replace the following with your actual connection details
username = 'user1'
password = 'IoP3X5_QRFiE'
servername = 'localhost'  # or your server's name
databasename = 'FoodFare'

# MSSQL connection URI
connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=ODBC+Driver+17+for+SQL+Server"
#connection_uri = f"mssql+pyodbc://{username}:{password}@{servername}/{databasename}?driver=SQL+Server"

db = SQLDatabase.from_uri(connection_uri, schemas=['dbo','Identity'])
print(db)

In [2]:
from langchain_community.agent_toolkits import (create_sql_agent, SQLDatabaseToolkit)
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
import autogen

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))
tools = toolkit.get_tools()

messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=SQL_FUNCTIONS_SUFFIX),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)



In [3]:
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True,
)

In [4]:
agent_executor.invoke({"input": "convert the Identity.Users table into a business object class in C#"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Identity.RoleAccess, Identity.RoleClaims, Identity.Roles, Identity.UserClaims, Identity.UserLogins, Identity.UserRoles, Identity.UserTokens, Identity.Users, dbo.Booths, dbo.UsersBooths, dbo.__EFMigrationsHistory
Invoking: `sql_db_schema` with `{'table_names': 'Identity.Users'}`



CREATE TABLE [Identity].[Users] (
	[Id] UNIQUEIDENTIFIER NOT NULL, 
	[Name] NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[UserName] NVARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[NormalizedUserName] NVARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Email] NVARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[NormalizedEmail] NVARCHAR(256) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[EmailConfirmed] BIT NOT NULL, 
	[PasswordHash] NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[SecurityStamp] NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Co

{'input': 'convert the Identity.Users table into a business object class in C#',
 'output': 'The `Identity.Users` table has the following schema:\n\n- Id: UNIQUEIDENTIFIER\n- Name: NVARCHAR(max)\n- UserName: NVARCHAR(256)\n- NormalizedUserName: NVARCHAR(256)\n- Email: NVARCHAR(256)\n- NormalizedEmail: NVARCHAR(256)\n- EmailConfirmed: BIT\n- PasswordHash: NVARCHAR(max)\n- SecurityStamp: NVARCHAR(max)\n- ConcurrencyStamp: NVARCHAR(max)\n- PhoneNumber: NVARCHAR(max)\n- PhoneNumberConfirmed: BIT\n- TwoFactorEnabled: BIT\n- LockoutEnd: DATETIMEOFFSET\n- LockoutEnabled: BIT\n- AccessFailedCount: INTEGER\n\nHere is a C# class representing the `Identity.Users` table as a business object:\n\n```csharp\nusing System;\n\npublic class User\n{\n    public Guid Id { get; set; }\n    public string Name { get; set; }\n    public string UserName { get; set; }\n    public string NormalizedUserName { get; set; }\n    public string Email { get; set; }\n    public string NormalizedEmail { get; set; }\n    